
## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [3]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=e13dd0325248c023b7e650b71cd613c3eed3a0299466becb0f74e2b747a8021e
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Your task

In [7]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [8]:
# Reading the text data into a dataframe
txt = spark.read.text("/content/pg100.txt")

In [9]:
#converting dataframe into rdd
rdd_txt = txt.rdd

In [10]:
#first 10 samples of rdd rows and their respective values
rdd_txt.collect()[:10 ]

[Row(value='The Project Gutenberg EBook of The Complete Works of William Shakespeare, by'),
 Row(value='William Shakespeare'),
 Row(value=''),
 Row(value='This eBook is for the use of anyone anywhere at no cost and with'),
 Row(value='almost no restrictions whatsoever.  You may copy it, give it away or'),
 Row(value='re-use it under the terms of the Project Gutenberg License included'),
 Row(value='with this eBook or online at www.gutenberg.org'),
 Row(value=''),
 Row(value='** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **'),
 Row(value='**     Please follow the copyright guidelines in this file.     **')]

In [11]:
#Converting it into a flat map and splitting the sentence into one word each
split_rdd_txt2 = rdd_txt.flatMap(lambda x: x['value'].split())

In [12]:
# Convert all words into lowercase letters and store the first letter
letters = split_rdd_txt2.map(lambda word: word[0].lower())

In [13]:
#mapping each letter to its value i.e. 1 right now
pairs = letters.map(lambda letter: (letter, 1))

In [14]:
pairs.collect()[:5]

[('t', 1), ('p', 1), ('g', 1), ('e', 1), ('o', 1)]

In [15]:
#reduce operation
from operator import add
counts = pairs.reduceByKey(add).collect()

In [16]:
# final result
counts

[('p', 27759),
 ('g', 20782),
 ('c', 34567),
 ('s', 65705),
 ('b', 45455),
 ('i', 62167),
 ('r', 14265),
 ('y', 25855),
 ('l', 29569),
 ('*', 24),
 ('d', 29713),
 ('1', 458),
 ('[', 2073),
 ('#', 3),
 ('j', 3339),
 ('h', 60563),
 ('.', 52),
 ('"', 356),
 ('9', 28),
 ('4', 46),
 ('_', 1),
 ('8', 15),
 ('?', 2),
 ('}', 2),
 ('$', 1),
 ('0', 6),
 ('t', 123602),
 ('e', 18697),
 ('o', 43494),
 ('w', 59597),
 ('f', 36814),
 ('u', 9170),
 ('a', 84836),
 ('n', 26759),
 ('m', 55676),
 ('2', 95),
 ('<', 248),
 ('v', 5728),
 ('(', 639),
 ('k', 9418),
 ('3', 59),
 ('/', 2),
 ("'", 3804),
 ('5', 35),
 ('q', 2377),
 ('6', 22),
 ('7', 17),
 ('z', 71),
 ('-', 52),
 (']', 7),
 ('x', 14),
 ('&', 21),
 (':', 1)]